**Notebook para la conversión de db sqlite to pg**

Bloque 0: Inicializacion - Definición de variables

*Importacion de librerias*

In [ ]:
import sqlalchemy as al
from configparser import ConfigParser
import pandas.io.sql as sqlio

*Configuraciones*

In [ ]:
# Funcion para la eExtraccion de parametros del archivo 'PGToPG-conf.ini' en secciones
def config(archivo='PGToPG-conf.ini', seccion='pg_server'):
    # Crear el parser y leer el archivo
    parser = ConfigParser()
    parser.read(archivo)
 
    # Obtener la sección de conexión a la base de datos
    result = {}
    if parser.has_section(seccion):
        params = parser.items(seccion)
        for param in params:
            result[param[0]] = param[1]
        return result
    else:
        raise Exception('Seccion {0} no encontrada en el archivo {1}'.format(seccion, archivo))

#params_pg_server = config() # Se extraen los parametros del servidor postgres de PGToPG-conf.ini
ExtractDB_params = config('PGToPG-conf.ini','Extract_from_pg_server') # Se extraen los parametros de la base postgres desde la cual se extraeran los datos
LoadDB_params = config('PGToPG-conf.ini','Load_to_pg_server') # Se extraen los parametros de la base postgres a la cual se cargaran los datos previamente extraidos
# No usar mayusculas para el nombre de la base de datos postgres

Bloque 1: Data extraction to Pandas Dataframes

In [ ]:
# Creamos una conexion en 'engine' para utilizar SQLAlchemy
dbConexionString = 'postgresql://'+ExtractDB_params['user']+':'+ExtractDB_params['password']+'@'+ExtractDB_params['host']+':'+ExtractDB_params['port']+'/' + ExtractDB_params['database']
engine = al.create_engine(dbConexionString)


# Definicion de dataframe 'tableNames' con los nombres de las tablas
tableNames = sqlio.read_sql_query("SELECT table_name\n\tFROM information_schema.tables\n\tWHERE table_schema="+"'"+ExtractDB_params['schema_name']+"'"+"\n\tAND table_type='BASE TABLE';", engine)
tablesToLoad = ["actions", "roles", "role_action", "user_roles"]
print(tableNames)
dbData = []
selectQuery = "SELECT * from "

# Iteracion en la que se carga 'dbData' con los datos de cada tabla de la BDD (un SELECT por tabla existente)
for i in range(len(tableNames)):
    currentTable = tableNames['table_name'][i]
    dbData.append(sqlio.read_sql_query(selectQuery + ExtractDB_params['schema_name'] + '.' + currentTable, engine))
    if (currentTable in tablesToLoad):
        print(dbData[i])

# Cerramos el engine de SQLAlchemy
engine.dispose()



Bloque 2: Carga de los datos

In [ ]:
# Creamos una conexion en 'engine' para utilizar SQLAlchemy
dbConexionString = 'postgresql://'+LoadDB_params['user']+':'+LoadDB_params['password']+'@'+LoadDB_params['host']+':'+LoadDB_params['port']+'/' + LoadDB_params['database']
engine = al.create_engine(dbConexionString)

with engine.begin() as connection:
    # Iteracion donde recorre cada dataframe en 'dbData' y carga los datos formateados de cada df a la tabla correspondiente en la nueva BDD
    for i in range(len(dbData)):
        if (tableNames['table_name'][i] in tablesToLoad):
            dbData[i].to_sql(tableNames['table_name'][i], con=connection, if_exists="append", index = False, schema=LoadDB_params['schema_name'],)

In [ ]:
# Cerramos el engine de SQLAlchemy
connection.close()
engine.dispose()